In [ ]:
#importing the dataset to be worked on = inaturalist dataset

%%capture
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip

train_dir='inaturalist_12K/train/'  #specifying the train and test data as per available in the dataset.
test_dir='inaturalist_12K/val/'

#specifying the classes as present in the iNaturalist dataset
 
categories=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

#so as we can see there are 10 classes in the inaturalist dataset.

In [ ]:
!pip install wandb

In [ ]:
#importing the libraries required
import numpy as np
import pandas as pd
import os
import keras
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation , BatchNormalization

#importing ImageDataGenarator for data augmentation 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2

In [ ]:
# importing the wandb dependencies. uncomment when using wandb

import wandb
from wandb.keras import WandbCallback

In [3]:
#importing the pretrained models from keras

from tensorflow.keras.applications.resnet50 import ResNet50 
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception 
from keras.applications.inception_v3 import InceptionV3 
from keras.applications.inception_resnet_v2 import InceptionResNetV2


In [5]:
# sweep. 
#run this cell when using wandb for sweep

sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "val_accuracy",
  "goal": "maximize"
  },
  'early_terminate': {
        'type':'hyperband',
        'min_iter': [3],
        's': [2]
  },
  "parameters": {
                    

        "base_model": {
            "values": [ "XCPTN", "IV3", "RN50", "IRV2"]
        },
        "epochs": {
            "values": [ 5, 10, 3]
        }, 
        "dense_neurons": {
            "values": [ 128, 256]
        },
        "batch_size": {
            "values": [ 16, 32]
        },
        "optimizer": {
            "values": [ "adam","nadam","rmsprop"]
        } 
              
    }
}


#this configuration gives the best validation accuracy

config_defaults={
    'fc_size':128,
    'batchnorm':'yes',
    'augmentation':'yes',
    'droprate':0.4,
    'pre_train':'inceptionresnetv2'
}

In [7]:
# defining the function for pretraining

def PreTrain():

  #initializing the wandb sweeps for combinations
  #uncomment it when to enable sweeps

  wandb.init(config=config_defaults)
  configs=wandb.config

  #initializing the sweeps with defaults set
  configs=config_defaults

  batchnorm=configs['batchnorm']
  augmentation=configs['augmentation']
  fc_size=configs['fc_size']
  droprate=configs['droprate']
  pre_train = configs['pre_train']

  #wandb.run.name='model_'+pre_train+'_fc_2_size_'+str(fc_size)+'_droprate_'+str(droprate)+ str('_bn_' if batchnorm=='yes' else '') +str('augment' if augmentation=='yes' else '')

  #setting up a flexible pretrained model 
  #assigning the weights already available as imagenet
  #the input image size as height*width*channels

  #the include_top part when we speify it as false, we chose not to include the fully connected layers and have more freedom on how to work with the model with a given possibility to retrain the weights of the model.

  if pre_train == 'inceptionv3':
    img_height = 299
    img_width = 299
    base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'inceptionresnetv2':
    img_height = 299
    img_width = 299
    base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'resnet50':
    img_height = 224
    img_width = 224
    base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'Xception':
    img_height = 299
    img_width = 299
    base_model = Xception(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))  


  for layers in base_model.layers:
    layers.trainable = False

  #setting up the flatten layer

  model = keras.Sequential([
      tf.keras.Input(shape=(img_height, img_width,3,)),
      base_model,
      Flatten(),
      Dense(fc_size,activation='relu'),
      
  ])

  #adding batch normalization
  if batchnorm == 'yes':
    model.add(BatchNormalization())
  
  #adding parameters to the model
  model.add(Dropout(droprate))
  model.add(Dense(fc_size, activation='relu'))
  model.add(Dropout(droprate))

  #adding the last layer with 10 classes and activation function as softmax
  
  model.add(Dense(10 ,activation='softmax'))
  

  #data generators for train(with and without validation) and validation
  augment= ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  train_set = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    color_mode='rgb',
    class_mode='sparse',
    shuffle=True,
    subset='training',
    seed=123,
  )

  aug_set = augment.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      batch_size=32,
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='training',
      seed=123,
  )

  val_set = datagen.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='validation',
      seed=123,
  )
 #compiling the whole model with parameters and associating the losses in the model as cross entropy
 
  model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
    metrics=['accuracy'],
  )

  #making the model flexible for augmentation
  # giving the callback for data plot as wandb
  if augmentation =='no' :
    hist=model.fit(train_set,epochs=2,validation_data=val_set,callbacks=[WandbCallback()]) 
    hist=model.fit(train_set,epochs=2,validation_data=val_set)
  
  else:
    hist=model.fit(aug_set,epochs=2,validation_data=val_set,callbacks=[WandbCallback()])
    hist=model.fit(aug_set,epochs=2,validation_data=val_set)

  #defining the validation accuracy
  val_acc=max(hist.history['val_accuracy'])
  params={'batch_norm':batchnorm,'augmentation':augmentation,'dropout':droprate,'pre_trained_model':pre_train,'val_acc':val_acc}
  #wandb.log(params)
  

In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#wandb sweep
#Ignore this and proceed
sweep_id=wandb.sweep(sweep_config,project="a-2", entity="dl22")
wandb.agent(sweep_id, PreTrain)   

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. {'s': [2], 'type': 'hyperband', 'min_iter': [3]} is not valid under any of the given schemas


Create sweep with ID: hixpo6a6
Sweep URL: https://wandb.ai/dl22/a-2/sweeps/hixpo6a6


wandb: Agent Starting Run: z4gnaoz9 with config:
wandb: 	base_model: IV3
wandb: 	batch_size: 16
wandb: 	dense_neurons: 128
wandb: 	epochs: 5
wandb: 	optimizer: adam
wandb: Currently logged in as: dl22 (use `wandb login --relogin` to force relogin)


219070464/219055592 [==============================] - 2s 0us/step
Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/2
282/282 [==============================] - 413s 1s/step - loss: 1.3366 - accuracy: 0.5821 - val_loss: 0.6756 - val_accuracy: 0.7968 - _timestamp: 1648980965.0000 - _runtime: 431.0000
Epoch 2/2
282/282 [==============================] - 382s 1s/step - loss: 0.9208 - accuracy: 0.7123 - val_loss: 0.5747 - val_accuracy: 0.8238 - _timestamp: 1648981347.0000 - _runtime: 813.0000
Epoch 1/2
282/282 [==============================] - 370s 1s/step - loss: 0.8272 - accuracy: 0.7407 - val_loss: 0.5588 - val_accuracy: 0.8298
Epoch 2/2
282/282 [==============================] - 370s 1s/step - loss: 0.7882 - accuracy: 0.7546 - val_loss: 0.6044 - val_accuracy: 0.8078



accuracy,▁█
dropout,▁
epoch,▁█
loss,█▁
val_acc,▁
val_accuracy,▁█
val_loss,█▁
accuracy,0.71233
augmentation,yes
batch_norm,yes
best_epoch,1


...

wandb: Agent Starting Run: 57g8nzvc with config:
wandb: 	base_model: IV3
wandb: 	batch_size: 16
wandb: 	dense_neurons: 128
wandb: 	epochs: 10
wandb: 	optimizer: adam


Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/2
282/282 [==============================] - 390s 1s/step - loss: 1.2865 - accuracy: 0.6014 - val_loss: 0.6291 - val_accuracy: 0.8088 - _timestamp: 1648990340.0000 - _runtime: 405.0000
Epoch 2/2
282/282 [==============================] - 366s 1s/step - loss: 0.8988 - accuracy: 0.7190 - val_loss: 0.6364 - val_accuracy: 0.7948 - _timestamp: 1648990708.0000 - _runtime: 773.0000
Epoch 1/2
282/282 [==============================] - 366s 1s/step - loss: 0.8088 - accuracy: 0.7490 - val_loss: 0.5981 - val_accuracy: 0.8278
Epoch 2/2
282/282 [==============================] - 367s 1s/step - loss: 0.7631 - accuracy: 0.7591 - val_loss: 0.5438 - val_accuracy: 0.8338



accuracy,▁█
dropout,▁
epoch,▁█
loss,█▁
val_acc,▁
val_accuracy,█▁
val_loss,▁█
accuracy,0.719
augmentation,yes
batch_norm,yes
best_epoch,0


wandb: Agent Starting Run: hi16vp0g with config:
wandb: 	base_model: IV3
wandb: 	batch_size: 16
wandb: 	dense_neurons: 256
wandb: 	epochs: 5
wandb: 	optimizer: adam


Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/2
282/282 [==============================] - 385s 1s/step - loss: 1.3118 - accuracy: 0.5859 - val_loss: 0.6399 - val_accuracy: 0.8018 - _timestamp: 1648991882.0000 - _runtime: 398.0000
Epoch 2/2
282/282 [==============================] - 364s 1s/step - loss: 0.8954 - accuracy: 0.7244 - val_loss: 0.5938 - val_accuracy: 0.8248 - _timestamp: 1648992246.0000 - _runtime: 762.0000
Epoch 1/2
103/282 [=========>....................] - ETA: 3:36 - loss: 0.8422 - accuracy: 0.7330

In [8]:
#Trial run with config defaults
PreTrain()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


219070464/219055592 [==============================] - 2s 0us/step
Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/2
282/282 [==============================] - 429s 1s/step - loss: 1.3598 - accuracy: 0.5811 - val_loss: 0.6286 - val_accuracy: 0.8068 - _timestamp: 1649498686.0000 - _runtime: 449.0000
Epoch 2/2
282/282 [==============================] - 394s 1s/step - loss: 0.9051 - accuracy: 0.7203 - val_loss: 0.6206 - val_accuracy: 0.8078 - _timestamp: 1649499080.0000 - _runtime: 843.0000
Epoch 1/2
282/282 [==============================] - 386s 1s/step - loss: 0.8187 - accuracy: 0.7422 - val_loss: 0.5532 - val_accuracy: 0.8208
Epoch 2/2
282/282 [==============================] - 386s 1s/step - loss: 0.7784 - accuracy: 0.7560 - val_loss: 0.5651 - val_accuracy: 0.8228


In [13]:
#not the correct one
#trial
#discard
#data generator for test set
wandb.init(config=config_defaults)
configs=config_defaults

batchnorm=configs['batchnorm']
droprate=configs['droprate']
augmentation=configs['augmentation']
fc_size=configs['fc_size']
pre_train = configs['pre_train']
 #wandb.run.name='model_'+pre_train+'_fc_2_size_'+str(fc_size)+'_droprate_'+str(droprate)+ str('_bn_' if batchnorm=='yes' else '') +str('augment' if augmentation=='yes' else '')

img_height = 299
img_width = 299
base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

model = keras.Sequential([
      tf.keras.Input(shape=(img_height, img_width,3,)),
      base_model,
      Flatten(),
      Dense(fc_size,activation='relu'),
      
  ])


model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(Dense(fc_size, activation='relu'))
model.add(Dropout(droprate))
model.add(Dense(10 ,activation='softmax'))


test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
  )

test_set = test_datagen.flow_from_directory(
      test_dir,
      target_size=(299,299),
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      seed=123,
  )

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
    metrics=['accuracy'],
  )


#evaluating model on test set
model.evaluate(test_set)

Found 2000 images belonging to 10 classes.
63/63 [==============================] - 55s 767ms/step - loss: 2.4637 - accuracy: 0.0855


[2.463688611984253, 0.08550000190734863]

In [ ]:
# Defining model with best configs to evaluate on test set  
#initializing the sweeps with defaults set
wandb.init(config=config_defaults)
configs=config_defaults

batchnorm=configs['batchnorm']
droprate=configs['droprate']
augmentation=configs['augmentation']
fc_size=configs['fc_size']
pre_train = configs['pre_train']

  #wandb.run.name='model_'+pre_train+'_fc_2_size_'+str(fc_size)+'_droprate_'+str(droprate)+ str('_bn_' if batchnorm=='yes' else '') +str('augment' if augmentation=='yes' else '')

#setting up a flexible pretrained model 
#assigning the weights already available as imagenet
#the input image size as height*width*channels
  
#the include_top part when we speify it as false, we chose not to include the fully connected layers and have more freedom on how to work with the model with a given possibility to retrain the weights of the model.

if pre_train == 'inceptionv3':
  img_height = 299
  img_width = 299
  base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

if pre_train == 'inceptionresnetv2':
  img_height = 299
  img_width = 299
  base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

if pre_train == 'resnet50':
  img_height = 224
  img_width = 224
  base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

if pre_train == 'Xception':
  img_height = 299
  img_width = 299
  base_model = Xception(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))  


for layers in base_model.layers:
  layers.trainable = False

#setting up the flatten layer
model = keras.Sequential([
      tf.keras.Input(shape=(img_height, img_width,3,)),
      base_model,
      Flatten(),
      Dense(fc_size,activation='relu'),
      
])

#adding batch normalization
if batchnorm == 'yes':
  model.add(BatchNormalization())

#adding parameters to the model
model.add(Dropout(droprate))
model.add(Dense(fc_size, activation='relu'))
model.add(Dropout(droprate))

#adding the last layer with 10 classes and activation function as softmax 
model.add(Dense(10 ,activation='softmax'))
  

#data generators for train(with and without validation) and validation
augment= ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
)

datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
)

train_set = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    color_mode='rgb',
    class_mode='sparse',
    shuffle=True,
    subset='training',
    seed=123,
)

aug_set = augment.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      batch_size=32,
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='training',
      seed=123,
)

val_set = datagen.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='validation',
      seed=123,
)

#compiling the whole model with parameters and associating the losses in the model as cross entropy
 
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
    metrics=['accuracy'],
)
  #making the model flexible for augmentation
  # giving the callback for data plot as wandb
if augmentation =='no' :
  hist=model.fit(train_set,epochs=2,validation_data=val_set,callbacks=[WandbCallback()]) 
  hist=model.fit(train_set,epochs=2,validation_data=val_set)
  
else:
  hist=model.fit(aug_set,epochs=2,validation_data=val_set,callbacks=[WandbCallback()])
  hist=model.fit(aug_set,epochs=2,validation_data=val_set)

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
  )

test_set = test_datagen.flow_from_directory(
      test_dir,
      target_size=(299,299),
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      seed=123,
  )
model.evaluate(test_set)